In [ ]:
import ipycytoscape

In [ ]:
from pathlib import Path

import wic
import sophios.cli
import sophios.plugins

args = sophios.cli.get_args()

tools_cwl = sophios.plugins.get_tools_cwl(args.homedir)
yml_paths = sophios.plugins.get_yml_paths(args.homedir)

In [ ]:
import graphviz
import networkx as nx
import yaml

import sophios.ast, sophios.utils, sophios.utils_graphs
from sophios.schemas import wic_schema
from sophios.utils_yaml import wic_loader
from sophios.wic_types import Yaml, YamlTree, GraphReps, NodeData, GraphData, StepId

yml_path_str = 'tutorial'
yml_namespace = 'global'
yml_path = yml_paths[yml_namespace][yml_path_str]

# Generate schemas for validation and vscode IntelliSense code completion
yaml_stems = sophios.utils.flatten([list(p) for p in yml_paths.values()])
validator = wic_schema.get_validator(tools_cwl, yaml_stems, write_to_disk=True)

# Load the high-level yaml workflow file.
with open(yml_path, mode='r', encoding='utf-8') as y:
    root_yaml_tree: Yaml = yaml.load(y.read(), Loader=wic_loader())
Path('autogenerated/').mkdir(parents=True, exist_ok=True)
wic_tag = {'wic': root_yaml_tree.get('wic', {})}
plugin_ns = wic_tag['wic'].get('namespace', 'global')
step_id = StepId(yml_path_str, plugin_ns)
y_t = YamlTree(step_id, root_yaml_tree)
yaml_tree_raw = sophios.ast.read_ast_from_disk(args.homedir, y_t, yml_paths, tools_cwl, validator)
with open(f'autogenerated/{Path(yml_path).stem}_tree_raw.wic', mode='w', encoding='utf-8') as f:
    f.write(yaml.dump(yaml_tree_raw.yml))
yaml_tree = sophios.ast.merge_yml_trees(yaml_tree_raw, {}, tools_cwl)
with open(f'autogenerated/{Path(yml_path).stem}_tree_merged.wic', mode='w', encoding='utf-8') as f:
    f.write(yaml.dump(yaml_tree.yml))
root_yml_dir_abs = Path(yml_path).parent.absolute()
yaml_tree = sophios.ast.python_script_generate_cwl(yaml_tree, root_yml_dir_abs, tools_cwl)
with open(f'autogenerated/{Path(yml_path).stem}_tree_python_script.wic', mode='w', encoding='utf-8') as f:
    f.write(yaml.dump(yaml_tree.yml))


yaml_forest = sophios.ast.tree_to_forest(yaml_tree, tools_cwl)
yaml_forest_lst =  sophios.utils.flatten_forest(yaml_forest)
yaml_subtrees_lst = [subforest.yaml_tree for subforest in yaml_forest_lst]
yaml_subtrees = {t.step_id.stem:t.yml for t in yaml_subtrees_lst}
#print(yaml_subtrees.keys())


graph_gv = graphviz.Digraph(name=f'cluster_{yml_path}')
graph_gv.attr(newrank='True')
graph_nx = nx.DiGraph()
graphdata = GraphData(yml_path)
graph = GraphReps(graph_gv, graph_nx, graphdata)
compiler_info = sophios.compiler.compile_workflow(yaml_tree, args, [], [graph], {}, {}, tools_cwl, True, relative_run_path=True, testing=False)
rose_tree = compiler_info.rose
sub_node_data: NodeData = rose_tree.data
yaml_stem = sub_node_data.name

In [ ]:
gd = sub_node_data.graph.graphdata
gd_flat = sophios.utils_graphs.flatten_graphdata(gd)
#print(len(gd_flat.nodes))
#print(len(gd_flat.edges))
json_cyto = sophios.utils_graphs.graphdata_to_cytoscape(gd_flat)
#import json
#json.dumps(json_cyto)

In [ ]:
cyto = ipycytoscape.CytoscapeWidget()

cyto.graph.add_graph_from_json(json_cyto, directed=True)

import json
with open("wicStyle.json", mode='r', encoding='utf-8') as f:
    s = json.load(f)

cyto.set_style(s)
cyto.set_layout(name='cola')
cyto.box_selection_enabled = False

In [ ]:
import ipywidgets
import yaml

textarea = ipywidgets.Textarea(layout=ipywidgets.Layout(height="auto", width="auto"))
out = ipywidgets.Output()

def node_name(node):
    textarea.description = node['data'].get('label', '')
    id = node['data'].get('id', '')
    #with out:
    #    print('id', id, id in yaml_subtrees)
    #    print(yaml_subtrees.get(id))
    textarea.value = yaml.dump(yaml_subtrees.get(id, ''))

# TODO: Round tripping: After user edits AST in textarea, re-compile.

cyto.on('node', 'click', node_name)

In [ ]:
from IPython.display import display
display(cyto, textarea)

In [ ]:
display(out)